In [ ]:
# install library as needed
# pip install chess

In [ ]:
import os
import datetime
import chess
import chess.engine
import chess.svg
import chess.pgn
import random
import numpy as np
import math
import pydot
from tqdm import tqdm
import io
import json
import graphviz
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display, SVG
from sklearn.preprocessing import MinMaxScaler
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

In [ ]:
# helper function: download file from Google Drive

def download_from_drive(file_name='lichess_db_eval.jsonl.zst', asset_dir='Mengqi_Input'):
    gauth = GoogleAuth()
    gauth.DEFAULT_SETTINGS['client_config_file'] = 'client_secret_1057507276332-5mk9ac9q22rsmtm1idlqvpraq08ar8p5.apps.googleusercontent.com.json'
    gauth.LoadCredentialsFile("mycreds.txt")
    if gauth.credentials is None:
        gauth.LocalWebserverAuth()
    elif gauth.access_token_expired:
        gauth.Refresh()
    else:
        gauth.Authorize()
    gauth.SaveCredentialsFile("mycreds.txt")
    drive = GoogleDrive(gauth)

    def find_folder_id(folder_name):
        file_list = drive.ListFile({'q': f"title='{folder_name}' and mimeType='application/vnd.google-apps.folder' and trashed=false"}).GetList()
        for file in file_list:
            if file['title'] == folder_name:
                return file['id']
        return None

    def download_zst_file_from_drive(file_title, parent_id):
        query = f"'{parent_id}' in parents and trashed=false and title='{file_title}'"
        file_list = drive.ListFile({'q': query}).GetList()
        if not file_list:
            print(f"No file found with title: {file_title}")
            return None
        file = file_list[0]
        print("loading file...") #3min
        file.GetContentFile(file_title)
        return file_title

    asset_folder_id = find_folder_id(asset_dir)
    if asset_folder_id is None:
        print("Asset folder not found.")
        return None

    file_path = download_zst_file_from_drive(file_name, asset_folder_id)
    if file_path is None:
        return None

    print("Downloaded file: {}".format(file_path))

In [ ]:
# download the stockfish engine for evaluation generation

download_from_drive(file_name='stockfish-windows-x86-64-avx2.exe', asset_dir='Mengqi_Input')

In [ ]:
# help functions

def stockfish(board, depth):
    with chess.engine.SimpleEngine.popen_uci("stockfish-windows-x86-64-avx2.exe") as sf:
        result = sf.analyse(board, chess.engine.Limit(depth=depth))
        score = result['score'].white().score()
    return score

# encode the legal moves
def board_encoder15(board):
    encoded_board = np.zeros([8,8,15]).astype(np.int8)
    fen = board.fen()
    fen_field = fen.split(' ')
    PiecePlacement = fen_field[0].split('/')
    piece_dict = {"R":0, "N":1, "B":2, "Q":3, "K":4, "P":5,
                  "r":6, "n":7, "b":8, "q":9, "k":10, "p":11
                 }
    for rank in range(8):
        pieces = ''
        for c in PiecePlacement[rank]:
            if c.isnumeric():
                pieces += '-'*int(c)
            else:
                pieces += c
        for file in range(8):
            if pieces[file] != '-':
                encoded_board[rank, file, piece_dict[pieces[file]]] = 1
    # plane 12 encodes all the legal moves of white
    aux = board.turn
    board.turn = chess.WHITE
    for move in board.legal_moves:
        encoded_board[7-np.unravel_index(move.to_square, (8,8))[0], np.unravel_index(move.to_square, (8,8))[1], 12] = 1
    # plane 13 encodes all the legal moves of black
    board.turn = chess.BLACK
    for move in board.legal_moves:
        encoded_board[7-np.unravel_index(move.to_square, (8,8))[0], np.unravel_index(move.to_square, (8,8))[1], 13] = 1
    board.turn = aux
    # plane 14 encodes the current player to move: white is 1, black is 0
    if fen_field[1] == 'w':
        encoded_board[:,:,14] = 1
    else:
        encoded_board[:,:,14] = 0
    return encoded_board

# does not encode the legal moves
def board_encoder13(board):
    encoded_board = np.zeros([8,8,13]).astype(np.int8)
    fen = board.fen()
    fen_field = fen.split(' ')
    PiecePlacement = fen_field[0].split('/')
    piece_dict = {"R":0, "N":1, "B":2, "Q":3, "K":4, "P":5,
                  "r":6, "n":7, "b":8, "q":9, "k":10, "p":11
                 }
    for rank in range(8):
        pieces = ''
        for c in PiecePlacement[rank]:
            if c.isnumeric():
                pieces += '-'*int(c)
            else:
                pieces += c
        for file in range(8):
            if pieces[file] != '-':
                encoded_board[rank, file, piece_dict[pieces[file]]] = 1
    # plane 12 encodes the current player to move: white is 1, black is 0
    if fen_field[1] == 'w':
        encoded_board[:,:,12] = 1
    else:
        encoded_board[:,:,12] = 0
    return encoded_board

In [ ]:
# model components

class InBlock(nn.Module):
    def __init__(self, channel_in=15, channel_out=256, kernel_size=3, stride=1):
        super(InBlock, self).__init__()
        self.channel_in = channel_in
        self.conv1 = nn.Conv2d(channel_in, channel_out, kernel_size=kernel_size,
                               stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(channel_out)

    def forward(self, s):
        if s.dtype != torch.float32:
            s = s.float()
        s = s.view(-1, self.channel_in, 8, 8)  # batch_size x channels x board_x x board_y
        s = self.conv1(s)
        s = self.bn1(s)
        s = F.relu(s)
        return s

class ConvBlock(nn.Module):
    def __init__(self, channel_in=256, channel_out=256, kernel_size=3, stride=1):
        super(ConvBlock, self).__init__()
        padding = (kernel_size - 1) // 2
        self.conv1 = nn.Conv2d(channel_in, channel_out, kernel_size=kernel_size,
                               stride=stride,
                     padding=padding, bias=False)
        self.bn1 = nn.BatchNorm2d(channel_out)
        self.conv2 = nn.Conv2d(channel_out, channel_out, kernel_size=kernel_size,
                               stride=stride,
                     padding=padding, bias=False)
        self.bn2 = nn.BatchNorm2d(channel_out)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = F.relu(out)
        return out

class ResBlock(nn.Module):
    def __init__(self, channel_in=256, channel_out=256, kernel_size=3, stride=1,
                 downsample=None):
        super(ResBlock, self).__init__()
        padding = (kernel_size - 1) // 2
        self.conv1 = nn.Conv2d(channel_in, channel_out, kernel_size=kernel_size,
                               stride=stride,
                     padding=padding, bias=False)
        self.bn1 = nn.BatchNorm2d(channel_out)
        self.conv2 = nn.Conv2d(channel_out, channel_out, kernel_size=kernel_size,
                               stride=stride,
                     padding=padding, bias=False)
        self.bn2 = nn.BatchNorm2d(channel_out)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample is not None:
            residual = self.downsample(x)
        out += residual
        out = F.relu(out)
        return out

class OutBlock(nn.Module):
    def __init__(self):
        super(OutBlock, self).__init__()
        self.conv = nn.Conv2d(256, 1, kernel_size=1, stride=1)
        self.bn = nn.BatchNorm2d(1)
        self.fc1 = nn.Linear(8*8, 256)
        self.fc2 = nn.Linear(256, 1)

    def forward(self,s):
        v = self.conv(s)
        v = self.bn(v)
        v = F.relu(v)
        v = v.view(-1, 8*8)  # batch_size x channels x board_x x board_y
        v = self.fc1(v)
        v = F.relu(v)
        v = self.fc2(v)
        v = torch.tanh(v)

        return v

In [ ]:
# 7 layers of residual blocks with legal moves encoded in input
class ResNet7_encode15(nn.Module):
    def __init__(self):
        super(ResNet7_encode15, self).__init__()
        self.conv = InBlock(channel_in=15)
        self.resblocks = nn.ModuleList([ResBlock() for _ in range(7)])
        self.outblock = OutBlock()

    def forward(self,s):
        s = self.conv(s)
        for resblock in self.resblocks:
            s = resblock(s)
        s = self.outblock(s)
        return s

# 13 layers of residual blocks with legal moves encoded in input
class ResNet13_encode15(nn.Module):
    def __init__(self):
        super(ResNet13_encode15, self).__init__()
        self.conv = InBlock(channel_in=15)
        self.resblocks = nn.ModuleList([ResBlock() for _ in range(13)])
        self.outblock = OutBlock()

    def forward(self,s):
        s = self.conv(s)
        for resblock in self.resblocks:
            s = resblock(s)
        s = self.outblock(s)
        return s

In [ ]:
# 7 layers of residual blocks with legal moves NOT encoded in input
class ResNet7_encode13(nn.Module):
    def __init__(self):
        super(ResNet7_encode13, self).__init__()
        self.conv = InBlock(channel_in=13)
        self.resblocks = nn.ModuleList([ResBlock() for _ in range(7)])
        self.outblock = OutBlock()

    def forward(self,s):
        s = self.conv(s)
        for resblock in self.resblocks:
            s = resblock(s)
        s = self.outblock(s)
        return s

# 13 layers of residual blocks with legal moves NOT encoded in input
class ResNet13_encode13(nn.Module):
    def __init__(self):
        super(ResNet13_encode13, self).__init__()
        self.conv = InBlock(channel_in=13)
        self.resblocks = nn.ModuleList([ResBlock() for _ in range(13)])
        self.outblock = OutBlock()

    def forward(self,s):
        s = self.conv(s)
        for resblock in self.resblocks:
            s = resblock(s)
        s = self.outblock(s)
        return s

# 1 layer of convolutional blocks with legal moves NOT encoded in input
class PlainNet1_encode13(nn.Module):
    def __init__(self):
        super(PlainNet1_encode13, self).__init__()
        self.conv = InBlock(channel_in=13)
        self.convblocks = nn.ModuleList([ConvBlock() for _ in range(1)])
        self.outblock = OutBlock()

    def forward(self,s):
        s = self.conv(s)
        for convblock in self.convblocks:
            s = convblock(s)
        s = self.outblock(s)
        return s

In [ ]:
def minimax_eval(board, model, move_encoded=True):
    if move_encoded:
        board_rep = board_encoder15(board)
    else:
        board_rep = board_encoder13(board)
    board_tensor = torch.tensor(board_rep.transpose(2,0,1), dtype=torch.float32)
    with torch.no_grad():
        output = model(board_tensor.unsqueeze(0))
    return output.item()

def minimax(board, model, depth, alpha, beta, move_encoded=True, maximizing=False):
    if depth == 0 or board.is_game_over():
        return minimax_eval(board, model, move_encoded)
    if maximizing: # maximizing the value
        max_val = -np.inf
        for move in board.legal_moves:
            board.push(move)
            val = minimax(board, model, depth - 1, alpha, beta, move_encoded, maximizing=False)
            board.pop()
            max_val = max(max_val, val)
            alpha = max(alpha, val)
            if beta <= alpha:
                break
        return max_val
    else:
        min_val = np.inf
        for move in board.legal_moves:
            board.push(move)
            val = minimax(board, model, depth - 1, alpha, beta, move_encoded, maximizing=True)
            board.pop()
            min_val = min(min_val, val)
            beta = min(beta, val)
            if beta <= alpha:
                break
        return min_val

def get_ai_move(board, model, player, move_encoded, depth):
    selected_move = None
    max_val = -np.inf
    min_val = np.inf

    for move in board.legal_moves:
        board.push(move)
        if player == chess.WHITE:
            val = minimax(board, model, depth - 1, -np.inf, np.inf, move_encoded, False)
            if val > max_val:
                max_val = val
                selected_move = move
        else:
            val = minimax(board, model, depth - 1, -np.inf, np.inf, move_encoded, True)
            if val < min_val:
                min_val = val
                selected_move = move
        board.pop()

    return selected_move

# **ResNet13 (white) vs. ResNet7 (black)**

In [ ]:
model_white = ResNet13_encode13()
model_black = ResNet7_encode13()
cuda = torch.cuda.is_available()
if cuda:
    model_white.cuda()
    checkpoint = torch.load("model/ResNet13_Mar28_training_checkpoint_epoch19.ckpt")
    model_white.load_state_dict(checkpoint['state_dict'])
    model_black.cuda()
    checkpoint = torch.load("model/ResNet7_Mar27_training_checkpoint_epoch20.ckpt")
    model_black.load_state_dict(checkpoint['state_dict'])
else:
    checkpoint = torch.load("model/ResNet13_Mar28_training_checkpoint_epoch19.ckpt", 
                            map_location=torch.device('cpu'))
    model_white.load_state_dict(checkpoint['state_dict'])
    checkpoint = torch.load("model/ResNet7_Mar27_training_checkpoint_epoch20.ckpt", 
                            map_location=torch.device('cpu'))
    model_black.load_state_dict(checkpoint['state_dict'])
model_white.eval()
model_black.eval()

In [ ]:
# play 15 games

for i in range(15):
    # Create a new game
    game = chess.pgn.Game()
    game.headers["Event"] = "Game Play"
    game.headers["Site"] = "Google Colab"
    game.headers["Date"] = datetime.datetime.now().strftime("%Y.%m.%d %H:%M:%S")
    game.headers["Round"] = str(i+1)
    game.headers["White"] = "ResNet13_Mar28"
    game.headers["Black"] = "ResNet7_Mar27"
    game_node = game
    pgn_file = "Arena/ResNet13_Mar28_vs_ResNet7_Mar26/game{}.pgn".format(i+1)

    board = chess.Board()
    for _ in range(10):
        all_moves = list(board.legal_moves)
        move = random.choice(all_moves)
        board.push(move)
        game_node = game_node.add_variation(move)

    while True:
        move = get_ai_move(board, model_white, chess.WHITE, move_encoded=False, depth=2)
        board.push(move)
        game_node = game_node.add_variation(move)

        if board.is_game_over():
            game.headers["Result"] = board.result()
            with open(pgn_file, "w") as f:
                exporter = chess.pgn.FileExporter(f)
                game.accept(exporter)
            print("Game saved to", pgn_file)
            break

        move = get_ai_move(board, model_black, chess.BLACK, move_encoded=False, depth=2)
        board.push(move)
        game_node = game_node.add_variation(move)

        if board.is_game_over():
            game.headers["Result"] = board.result()
            with open(pgn_file, "w") as f:
                exporter = chess.pgn.FileExporter(f)
                game.accept(exporter)
            print("Game saved to", pgn_file)
            break

In [ ]:
for i in range(15):
    game = chess.pgn.read_game(open("Arena/ResNet13_Mar28_vs_ResNet7_Mar26/game{}.pgn".format(i+1)))
    print(game.headers["Result"])

# **ResNet7 (white) vs. ResNet13 (black)**

In [ ]:
model_white = ResNet7_encode13()
model_black = ResNet13_encode13()
cuda = torch.cuda.is_available()
if cuda:
    model_white.cuda()
    checkpoint = torch.load("model/ResNet7_Mar27_training_checkpoint_epoch20.ckpt")
    model_white.load_state_dict(checkpoint['state_dict'])
    model_black.cuda()
    checkpoint = torch.load("model/ResNet13_Mar28_training_checkpoint_epoch19.ckpt")
    model_black.load_state_dict(checkpoint['state_dict'])
else:
    checkpoint = torch.load("model/ResNet7_Mar27_training_checkpoint_epoch20.ckpt", 
                            map_location=torch.device('cpu'))
    model_white.load_state_dict(checkpoint['state_dict'])
    checkpoint = torch.load("model/ResNet13_Mar28_training_checkpoint_epoch19.ckpt", 
                            map_location=torch.device('cpu'))
    model_black.load_state_dict(checkpoint['state_dict'])
model_white.eval()
model_black.eval()

In [ ]:
# play 15 games

for i in range(15):
    # Create a new game
    game = chess.pgn.Game()
    game.headers["Event"] = "Game Play"
    game.headers["Site"] = "Google Colab"
    game.headers["Date"] = datetime.datetime.now().strftime("%Y.%m.%d %H:%M:%S")
    game.headers["Round"] = str(i+1)
    game.headers["White"] = "ResNet7_Mar27"
    game.headers["Black"] = "ResNet13_Mar28"
    game_node = game
    pgn_file = "Arena/ResNet7_Mar26_vs_ResNet13_Mar28/game{}.pgn".format(i+1)

    board = chess.Board()
    for _ in range(10):
        all_moves = list(board.legal_moves)
        move = random.choice(all_moves)
        board.push(move)
        game_node = game_node.add_variation(move)

    while True:
        move = get_ai_move(board, model_white, chess.WHITE, move_encoded=False, depth=2)
        board.push(move)
        game_node = game_node.add_variation(move)

        if board.is_game_over():
            game.headers["Result"] = board.result()
            with open(pgn_file, "w") as f:
                exporter = chess.pgn.FileExporter(f)
                game.accept(exporter)
            print("Game saved to", pgn_file)
            break

        move = get_ai_move(board, model_black, chess.BLACK, move_encoded=False, depth=2)
        board.push(move)
        game_node = game_node.add_variation(move)

        if board.is_game_over():
            game.headers["Result"] = board.result()
            with open(pgn_file, "w") as f:
                exporter = chess.pgn.FileExporter(f)
                game.accept(exporter)
            print("Game saved to", pgn_file)
            break

In [ ]:
for i in range(15):
    game = chess.pgn.read_game(open("Arena/ResNet7_Mar26_vs_ResNet13_Mar28/game{}.pgn".format(i+1)))
    print(game.headers["Result"])

# **PlainNet1 (white) vs. ResNet7 (black)**

In [ ]:
model_white = PlainNet1_encode13()
model_black = ResNet7_encode13()
cuda = torch.cuda.is_available()
if cuda:
    model_white.cuda()
    checkpoint = torch.load("model/PlainNet1_Mar31_training_checkpoint_epoch28.ckpt")
    model_white.load_state_dict(checkpoint['state_dict'])
    model_black.cuda()
    checkpoint = torch.load("model/ResNet7_Mar27_training_checkpoint_epoch20.ckpt")
    model_black.load_state_dict(checkpoint['state_dict'])
else:
    checkpoint = torch.load("model/PlainNet1_Mar31_training_checkpoint_epoch28.ckpt", 
                            map_location=torch.device('cpu'))
    model_white.load_state_dict(checkpoint['state_dict'])
    checkpoint = torch.load("model/ResNet7_Mar27_training_checkpoint_epoch20.ckpt", 
                            map_location=torch.device('cpu'))
    model_black.load_state_dict(checkpoint['state_dict'])
model_white.eval()
model_black.eval()

In [ ]:
# play 15 games

for i in range(15):
    # Create a new game
    game = chess.pgn.Game()
    game.headers["Event"] = "Game Play"
    game.headers["Site"] = "Google Colab"
    game.headers["Date"] = datetime.datetime.now().strftime("%Y.%m.%d %H:%M:%S")
    game.headers["Round"] = str(i+1)
    game.headers["White"] = "PlainNet1_Mar31"
    game.headers["Black"] = "ResNet7_Mar26"
    game_node = game
    pgn_file = "Arena/PlainNet1_Mar31_vs_ResNet7_Mar26/game{}.pgn".format(i+1)

    board = chess.Board()
    for _ in range(10):
        all_moves = list(board.legal_moves)
        move = random.choice(all_moves)
        board.push(move)
        game_node = game_node.add_variation(move)

    while True:
        move = get_ai_move(board, model_white, chess.WHITE, move_encoded=False, depth=2)
        board.push(move)
        game_node = game_node.add_variation(move)
    
        if board.is_game_over():
            game.headers["Result"] = board.result()
            with open(pgn_file, "w") as f:
                exporter = chess.pgn.FileExporter(f)
                game.accept(exporter)
            print("Game saved to", pgn_file)
            break

        move = get_ai_move(board, model_black, chess.BLACK, move_encoded=False, depth=2)
        board.push(move)
        game_node = game_node.add_variation(move)

        if board.is_game_over():
            game.headers["Result"] = board.result()
            with open(pgn_file, "w") as f:
                exporter = chess.pgn.FileExporter(f)
                game.accept(exporter)
            print("Game saved to", pgn_file)
            break

In [ ]:
for i in range(15):
    game = chess.pgn.read_game(open("Arena/PlainNet1_Mar31_vs_ResNet7_Mar26/game{}.pgn".format(i+1)))
    print(game.headers["Result"])

# **ResNet7 (white) vs. PlainNet1 (black)**

In [ ]:
model_white = ResNet7_encode13()
model_black = PlainNet1_encode13()
cuda = torch.cuda.is_available()
if cuda:
    model_white.cuda()
    checkpoint = torch.load("model/ResNet7_Mar27_training_checkpoint_epoch20.ckpt")
    model_white.load_state_dict(checkpoint['state_dict'])
    model_black.cuda()
    checkpoint = torch.load("model/PlainNet1_Mar31_training_checkpoint_epoch28.ckpt")
    model_black.load_state_dict(checkpoint['state_dict'])
else:
    checkpoint = torch.load("model/ResNet7_Mar27_training_checkpoint_epoch20.ckpt", 
                          map_location=torch.device('cpu'))
    model_white.load_state_dict(checkpoint['state_dict'])
    checkpoint = torch.load("model/PlainNet1_Mar31_training_checkpoint_epoch28.ckpt", 
                          map_location=torch.device('cpu'))
    model_black.load_state_dict(checkpoint['state_dict'])
model_white.eval()
model_black.eval()

In [ ]:
# play 15 games

for i in range(15):
    # Create a new game
    game = chess.pgn.Game()
    game.headers["Event"] = "Game Play"
    game.headers["Site"] = "Google Colab"
    game.headers["Date"] = datetime.datetime.now().strftime("%Y.%m.%d %H:%M:%S")
    game.headers["Round"] = str(i+1)
    game.headers["White"] = "ResNet7_Mar26"
    game.headers["Black"] = "PlainNet1_Mar31"
    game_node = game
    pgn_file = "Arena/ResNet7_Mar26_vs_PlainNet1_Mar31/game{}.pgn".format(i+1)

    board = chess.Board()
    for _ in range(10):
        all_moves = list(board.legal_moves)
        move = random.choice(all_moves)
        board.push(move)
        game_node = game_node.add_variation(move)

    while True:
        move = get_ai_move(board, model_white, chess.WHITE, move_encoded=False, depth=2)
        board.push(move)
        game_node = game_node.add_variation(move)

        if board.is_game_over():
            game.headers["Result"] = board.result()
            with open(pgn_file, "w") as f:
                exporter = chess.pgn.FileExporter(f)
                game.accept(exporter)
            print("Game saved to", pgn_file)
            break

        move = get_ai_move(board, model_black, chess.BLACK, move_encoded=False, depth=2)
        board.push(move)
        game_node = game_node.add_variation(move)
        
        if board.is_game_over():
            game.headers["Result"] = board.result()
            with open(pgn_file, "w") as f:
                exporter = chess.pgn.FileExporter(f)
                game.accept(exporter)
            print("Game saved to", pgn_file)
            break

In [ ]:
for i in range(15):
    game = chess.pgn.read_game(open("Arena/ResNet7_Mar26_vs_PlainNet1_Mar31/game{}.pgn".format(i+1)))
    print(game.headers["Result"])

# **PlainNet1 (white) vs. ResNet13 (black)**

In [ ]:
model_white = PlainNet1_encode13()
model_black = ResNet13_encode13()
cuda = torch.cuda.is_available()
if cuda:
    model_white.cuda()
    checkpoint = torch.load("model/PlainNet1_Mar31_training_checkpoint_epoch28.ckpt")
    model_white.load_state_dict(checkpoint['state_dict'])
    model_black.cuda()
    checkpoint = torch.load("model/ResNet13_Mar28_training_checkpoint_epoch19.ckpt")
    model_black.load_state_dict(checkpoint['state_dict'])
else:
    checkpoint = torch.load("model/PlainNet1_Mar31_training_checkpoint_epoch28.ckpt", 
                            map_location=torch.device('cpu'))
    model_white.load_state_dict(checkpoint['state_dict'])
    checkpoint = torch.load("model/ResNet13_Mar28_training_checkpoint_epoch19.ckpt", 
                            map_location=torch.device('cpu'))
    model_black.load_state_dict(checkpoint['state_dict'])
model_white.eval()
model_black.eval()

In [ ]:
# play 15 games

for i in range(15):
    # Create a new game
    game = chess.pgn.Game()
    game.headers["Event"] = "Game Play"
    game.headers["Site"] = "Google Colab"
    game.headers["Date"] = datetime.datetime.now().strftime("%Y.%m.%d %H:%M:%S")
    game.headers["Round"] = str(i+1)
    game.headers["White"] = "PlainNet1_Mar31"
    game.headers["Black"] = "ResNet13_Mar28"
    game_node = game
    pgn_file = "Arena/PlainNet1_Mar31_vs_ResNet13_Mar28/game{}.pgn".format(i+1)

    board = chess.Board()
    for _ in range(10):
        all_moves = list(board.legal_moves)
        move = random.choice(all_moves)
        board.push(move)
        game_node = game_node.add_variation(move)

    while True:
        move = get_ai_move(board, model_white, chess.WHITE, move_encoded=False, depth=2)
        board.push(move)
        game_node = game_node.add_variation(move)

        if board.is_game_over():
            game.headers["Result"] = board.result()
            with open(pgn_file, "w") as f:
                exporter = chess.pgn.FileExporter(f)
                game.accept(exporter)
            print("Game saved to", pgn_file)
            break

        move = get_ai_move(board, model_black, chess.BLACK, move_encoded=False, depth=2)
        board.push(move)
        game_node = game_node.add_variation(move)

        if board.is_game_over():
            game.headers["Result"] = board.result()
            with open(pgn_file, "w") as f:
                exporter = chess.pgn.FileExporter(f)
                game.accept(exporter)
            print("Game saved to", pgn_file)
            break

In [ ]:
for i in range(15):
    game = chess.pgn.read_game(open("Arena/PlainNet1_Mar31_vs_ResNet13_Mar28/game{}.pgn".format(i+1)))
    print(game.headers["Result"])

# **ResNet13 (white) vs. PlainNet1 (black)**

In [ ]:
model_white = ResNet13_encode13()
model_black = PlainNet1_encode13()
cuda = torch.cuda.is_available()
if cuda:
    model_white.cuda()
    checkpoint = torch.load("model/ResNet13_Mar28_training_checkpoint_epoch19.ckpt")
    model_white.load_state_dict(checkpoint['state_dict'])
    model_black.cuda()
    checkpoint = torch.load("model/PlainNet1_Mar31_training_checkpoint_epoch28.ckpt")
    model_black.load_state_dict(checkpoint['state_dict'])
else:
    checkpoint = torch.load("model/ResNet13_Mar28_training_checkpoint_epoch19.ckpt", 
                            map_location=torch.device('cpu'))
    model_white.load_state_dict(checkpoint['state_dict'])
    checkpoint = torch.load("model/PlainNet1_Mar31_training_checkpoint_epoch28.ckpt", 
                            map_location=torch.device('cpu'))
    model_black.load_state_dict(checkpoint['state_dict'])
model_white.eval()
model_black.eval()

In [ ]:
# play 15 games

for i in range(15):
    # Create a new game
    game = chess.pgn.Game()
    game.headers["Event"] = "Game Play"
    game.headers["Site"] = "Google Colab"
    game.headers["Date"] = datetime.datetime.now().strftime("%Y.%m.%d %H:%M:%S")
    game.headers["Round"] = str(i+1)
    game.headers["White"] = "ResNet13_Mar28"
    game.headers["Black"] = "PlainNet1_Mar31"
    game_node = game
    pgn_file = "Arena/ResNet13_Mar28_vs_PlainNet1_Mar31/game{}.pgn".format(i+1)

    board = chess.Board()
    for _ in range(10):
        all_moves = list(board.legal_moves)
        move = random.choice(all_moves)
        board.push(move)
        game_node = game_node.add_variation(move)

    while True:
        move = get_ai_move(board, model_white, chess.WHITE, move_encoded=False, depth=2)
        board.push(move)
        game_node = game_node.add_variation(move)

        if board.is_game_over():
            game.headers["Result"] = board.result()
            with open(pgn_file, "w") as f:
                exporter = chess.pgn.FileExporter(f)
                game.accept(exporter)
            print("Game saved to", pgn_file)
            break

        move = get_ai_move(board, model_black, chess.BLACK, move_encoded=False, depth=2)
        board.push(move)
        game_node = game_node.add_variation(move)

        if board.is_game_over():
            game.headers["Result"] = board.result()
            with open(pgn_file, "w") as f:
                exporter = chess.pgn.FileExporter(f)
                game.accept(exporter)
            print("Game saved to", pgn_file)
            break

In [ ]:
for i in range(15):
    game = chess.pgn.read_game(open("Arena/ResNet13_Mar28_vs_PlainNet1_Mar31/game{}.pgn".format(i+1)))
    print(game.headers["Result"])

# **ResNet13 vs. stockfish**
50 games, 25 games each with one player as white and another as black

In [ ]:
model_white = ResNet13_encode13()
model_black = ResNet13_encode13()
cuda = torch.cuda.is_available()
if cuda:
    model_white.cuda()
    checkpoint = torch.load("model/ResNet13_Mar28_training_checkpoint_epoch19.ckpt")
    model_white.load_state_dict(checkpoint['state_dict'])
    model_black.cuda()
    checkpoint = torch.load("model/ResNet13_Mar28_training_checkpoint_epoch19.ckpt")
    model_black.load_state_dict(checkpoint['state_dict'])
else:
    checkpoint = torch.load("model/ResNet13_Mar28_training_checkpoint_epoch19.ckpt", 
                            map_location=torch.device('cpu'))
    model_white.load_state_dict(checkpoint['state_dict'])
    checkpoint = torch.load("model/ResNet13_Mar28_training_checkpoint_epoch19.ckpt", 
                            map_location=torch.device('cpu'))
    model_black.load_state_dict(checkpoint['state_dict'])
model_white.eval()
model_black.eval()

In [ ]:
# play 25 games

for i in range(25):
    # Create a new game
    game = chess.pgn.Game()
    game.headers["Event"] = "Game Play"
    game.headers["Site"] = "Google Colab"
    game.headers["Date"] = datetime.datetime.now().strftime("%Y.%m.%d %H:%M:%S")
    game.headers["Round"] = str(i+1)
    game.headers["White"] = "ResNet13_Mar28"
    game.headers["Black"] = "stockfish"
    game_node = game
    pgn_file = "Arena/ResNet13_Mar28_vs_stockfish/game{}.pgn".format(i+1)

    board = chess.Board()
    for _ in range(10):
        all_moves = list(board.legal_moves)
        move = random.choice(all_moves)
        board.push(move)
        game_node = game_node.add_variation(move)

    with chess.engine.SimpleEngine.popen_uci("stockfish-windows-x86-64-avx2.exe") as sf:
        while True:
            move = get_ai_move(board, model_white, chess.WHITE, move_encoded=False, depth=2)
            board.push(move)
            game_node = game_node.add_variation(move)

            if board.is_game_over():
                game.headers["Result"] = board.result()
                with open(pgn_file, "w") as f:
                    exporter = chess.pgn.FileExporter(f)
                    game.accept(exporter)
                print("Game saved to", pgn_file)
                break

            move = sf.analyse(board, chess.engine.Limit(depth=3))['pv'][0]
            board.push(move)
            game_node = game_node.add_variation(move)
            
            if board.is_game_over():
                game.headers["Result"] = board.result()
                with open(pgn_file, "w") as f:
                    exporter = chess.pgn.FileExporter(f)
                    game.accept(exporter)
                print("Game saved to", pgn_file)
                break

In [ ]:
# play 25 games

for i in range(25):
    # Create a new game
    game = chess.pgn.Game()
    game.headers["Event"] = "Game Play"
    game.headers["Site"] = "Google Colab"
    game.headers["Date"] = datetime.datetime.now().strftime("%Y.%m.%d %H:%M:%S")
    game.headers["Round"] = str(i+1)
    game.headers["White"] = "stockfish"
    game.headers["Black"] = "ResNet13_Mar28"
    game_node = game
    pgn_file = "Arena/stockfish_vs_ResNet13_Mar28/game{}.pgn".format(i+1)

    board = chess.Board()
    for _ in range(10):
        all_moves = list(board.legal_moves)
        move = random.choice(all_moves)
        board.push(move)
        game_node = game_node.add_variation(move)

    with chess.engine.SimpleEngine.popen_uci("stockfish-windows-x86-64-avx2.exe") as sf:
        while True:
            move = sf.analyse(board, chess.engine.Limit(depth=3))['pv'][0]
            board.push(move)
            game_node = game_node.add_variation(move)

            if board.is_game_over():
                game.headers["Result"] = board.result()
                with open(pgn_file, "w") as f:
                    exporter = chess.pgn.FileExporter(f)
                    game.accept(exporter)
                print("Game saved to", pgn_file)
                break

            move = get_ai_move(board, model_black, chess.BLACK, move_encoded=False, depth=2)
            board.push(move)
            game_node = game_node.add_variation(move)

            if board.is_game_over():
                game.headers["Result"] = board.result()
                with open(pgn_file, "w") as f:
                    exporter = chess.pgn.FileExporter(f)
                    game.accept(exporter)
                print("Game saved to", pgn_file)
                break